<a href="https://colab.research.google.com/github/tbrar/CPSC-501-A4-TensorFlow/blob/master/CHDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [36]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test (1).csv


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [0]:
train_file_path = "heart_train.csv"
test_file_path = "heart_test.csv"

In [39]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

SELECT_COLUMNS = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age', 'chd']
DEFAULTS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

example_batch, labels_batch = next(iter(temp_dataset)) 
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

example_batch, labels_batch = next(iter(temp_dataset)) 



[[1.260e+02 3.100e+00 2.010e+00 3.297e+01 5.600e+01 2.863e+01 2.674e+01
  4.500e+01]
 [1.300e+02 8.000e-02 5.590e+00 2.542e+01 5.000e+01 2.498e+01 6.270e+00
  4.300e+01]
 [1.340e+02 2.750e+00 5.510e+00 2.617e+01 5.700e+01 2.987e+01 8.330e+00
  3.300e+01]
 [1.480e+02 4.800e+00 6.090e+00 3.655e+01 6.300e+01 2.544e+01 8.800e-01
  5.500e+01]
 [1.440e+02 4.000e-01 4.640e+00 3.009e+01 3.000e+01 2.739e+01 7.400e-01
  5.500e+01]]

[0 1 0 1 0]


In [40]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['sbp', 'tobacco', 'ldl', 'adiposity', 'typea', 'obesity', 'alcohol', 'age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

example_batch, labels_batch = next(iter(packed_train_data)) 

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,139.088384,3.534470,4.721641,25.417854,52.611111,25.980404,17.155328,42.727273
std,20.883153,4.622259,1.969109,7.862184,9.908200,4.185559,24.696052,14.758524
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000
25%,125.500000,0.050000,3.300000,19.825000,46.000000,22.757500,0.510000,31.000000
50%,134.000000,1.800000,4.325000,26.115000,53.000000,25.780000,7.780000,44.500000
75%,148.500000,5.152500,5.632500,31.102500,59.000000,28.402500,23.737500,55.250000
max,218.000000,31.200000,14.160000,42.490000,77.000000,45.720000,147.190000,64.000000


In [41]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()


array([[ 1.0013632e+00, -6.3485622e-01,  1.1265801e+00,  6.5021962e-01,
        -1.6765013e+00, -7.1771443e-02, -6.3715965e-01,  1.4413859e+00],
       [ 3.2998664e+00,  5.1631751e+00, -8.1338412e-01,  1.5417406e-01,
         1.3512938e+00,  3.5111126e-01,  6.4442164e-01,  1.3058710e+00],
       [-4.3520147e-01, -7.4735534e-01,  4.4099072e-01,  2.7292254e-04,
        -2.6353028e-01, -2.3901333e-01, -4.4077200e-01,  1.8479353e-02],
       [-1.4788853e-01, -4.4014621e-01,  6.7967731e-01,  1.4272708e-01,
         1.4017574e-01,  8.1222016e-01, -1.0752037e-01, -6.5909517e-01],
       [-7.2251439e-01,  2.7811739e-01, -7.5244278e-01, -5.4919267e-01,
        -4.6538326e-01,  5.9958440e-01, -3.5371357e-01, -8.6236757e-01]],
      dtype=float32)

In [42]:
CATEGORIES = {
    'famhist': ['Absent', 'Present'],
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

# See what you just created.
categorical_columns

categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])


[1. 0.]


In [43]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])


[ 1.          0.          1.0013632  -0.6348562   1.1265801   0.6502196
 -1.6765013  -0.07177144 -0.63715965  1.4413859 ]


In [55]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(500, kernel_regularizer=tf.keras.regularizers.l2(0.05), activation='relu'),
  tf.keras.layers.Dense(300, kernel_regularizer=tf.keras.regularizers.l2(0.05), activation='relu'),
  #tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(100, kernel_regularizer=tf.keras.regularizers.l2(0.05), activation='relu'),
  #tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

model.fit(train_data, epochs=20)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

predictions = model.predict(test_data)

# Show some results
for prediction, chd in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted chd: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Has CHD" if bool(chd) else "Doesn't have CHD"))


Epoch 1/20
80/80 [==============================] - 2s 21ms/step - loss: 9.9583 - accuracy: 0.6641
Epoch 2/20
80/80 [==============================] - 1s 7ms/step - loss: 1.0268 - accuracy: 0.6717
Epoch 3/20
80/80 [==============================] - 1s 7ms/step - loss: 0.6616 - accuracy: 0.6667
Epoch 4/20
80/80 [==============================] - 1s 6ms/step - loss: 0.6400 - accuracy: 0.7045
Epoch 5/20
80/80 [==============================] - 0s 6ms/step - loss: 0.6337 - accuracy: 0.6667
Epoch 6/20
80/80 [==============================] - 0s 6ms/step - loss: 0.6237 - accuracy: 0.6692
Epoch 7/20
80/80 [==============================] - 1s 7ms/step - loss: 0.6263 - accuracy: 0.6869
Epoch 8/20
80/80 [==============================] - 1s 7ms/step - loss: 0.6196 - accuracy: 0.7222
Epoch 9/20
80/80 [==============================] - 1s 7ms/step - loss: 0.6212 - accuracy: 0.7197
Epoch 10/20
80/80 [==============================] - 1s 7ms/step - loss: 0.6061 - accuracy: 0.7071
Epoch 11/20
80/80 

In [0]:
#